In [ ]:
%load_ext autoreload
%autoreload 2

: 

In [ ]:
#!pip install openvino-dev
#!pip install onnxruntime

In [ ]:
from IPython.display import Markdown, display
import onnxruntime

In [ ]:
import sys,os
import torch
sys.path.append(os.path.abspath('../models'))
from tinyism import tinyModel

In [ ]:
posencoding=True
model=tinyModel(posEncoding=posencoding)

In [ ]:
 weight="../weights/resnet18_inst.pth"
 model.load_state_dict(torch.load(weight,map_location=torch.device('cpu')))
 model.eval()

In [ ]:
    sys.path.append(os.path.abspath('../datasets'))
    from data_loader import preprocess
    import cv2
    img=cv2.imread("../test3.png")
    img=cv2.resize(img,(512,512))
    x=preprocess((torch.from_numpy(img).float())).unsqueeze(0)

: 

In [ ]:
torchout=model(x)
x.shape,torchout[0].shape,torchout[1].shape

In [ ]:
input_names=['images']
output_names = ['output0', 'output1'] 
dynamic = {'images': {0: 'batch'}}  
dynamic['output0'] = {0: 'instances'} 
dynamic['output1'] = {0: 'instances'}  # shape(1,insts,64,64)


torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "tinynet.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = input_names,   # the model's input names
                  output_names = output_names, # the model's output names
                  dynamic_axes=dynamic
                  )

In [ ]:
ort_session = onnxruntime.InferenceSession("tinynet.onnx")

In [ ]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [ ]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

In [ ]:
ort_outs[0].shape,ort_outs[1].shape

In [ ]:
input_image=to_numpy(x)
torchout1=to_numpy(torchout[0])
torchout2=to_numpy(torchout[1])
ort_outs[0].shape,ort_outs[1].shape,torchout1.shape,torchout2.shape

In [ ]:
import numpy as np
np.allclose(torchout1, ort_outs[0], rtol=1e-03, atol=1e-05),np.allclose(torchout2, ort_outs[1], rtol=1e-03, atol=1e-05)

In [ ]:
# Construct the command for Model Optimizer.
from pathlib import Path
onnx_path="tinynet.onnx"
model_path="irmodel"
model_path = Path(model_path)
ir_path = model_path.with_suffix(".xml")
IMAGE_HEIGHT=512
IMAGE_WIDTH=512
mo_command = f"""mo
                 --input_model "{onnx_path}"
                 --input_shape "[1,3, {IMAGE_HEIGHT}, {IMAGE_WIDTH}]"
                 --data_type FP32
                 --output_dir "{model_path}"
                 """
mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert the ONNX model to OpenVINO:")
display(Markdown(f"`{mo_command}`"))

In [ ]:
print("Exporting ONNX model to IR... This may take a few minutes.")
mo_result = %sx $mo_command
print("\n".join(mo_result))

In [ ]:
ir_path=Path("irmodel/tinynet.xml")

In [ ]:
from openvino.runtime import Core
ie = Core()

In [ ]:
model_ir = ie.read_model(model=ir_path)
compiled_model_ir = ie.compile_model(model=model_ir, device_name="CPU")

In [ ]:
# Get input and output layers.
output_layer_ir = compiled_model_ir.output(1)
output_layer_ir

In [ ]:
res_ir = compiled_model_ir([input_image])[output_layer_ir]
res_ir.shape

In [ ]:
np.allclose(res_ir,torchout2,rtol=1e-04, atol=1e-05)